### Import Library

In [6]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re


### Load Dataset

In [7]:
faq_data = pd.read_excel('dataset.xlsx')
faq_data.head()

,Pertanyaan,Jawaban
0,Apa saja lomba yang bisa saya ikuti?,Berikut adalah beberapa lomba yang dapat Anda ...
1,Lomba apa saja yang dapat saya ikuti sebagai m...,Beberapa lomba yang dapat Anda ikuti sebagai m...
2,Apa saja kompetisi yang bisa diikuti oleh maha...,Anda dapat mengikuti berbagai lomba seperti PI...
3,Lomba apa saja yang diadakan untuk mahasiswa d...,Beberapa lomba yang diadakan untuk mahasiswa a...
4,Lomba-lomba apa yang tersedia untuk mahasiswa?,Lomba-lomba yang tersedia untuk mahasiswa anta...


### Inisialisasi Stemmer Indonesia dari Sastrawi

In [8]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

### Melakukan preprocessing pada pertanyaan di dataset

In [9]:
def preprocess_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    processed_text = [stemmer.stem(word) for word in words]
    return ' '.join(processed_text)

faq_data['Processed_Pertanyaan'] = faq_data['Pertanyaan'].apply(preprocess_text)

### TF-IDF Vectorization

In [10]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(faq_data['Processed_Pertanyaan'])

### Membuat Fungsi Respon

In [11]:
def get_response(user_query):
    processed_query = preprocess_text(user_query)
    query_tfidf = vectorizer.transform([processed_query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)
    most_similar_idx = np.argmax(cosine_similarities)
    response = faq_data.iloc[most_similar_idx]['Jawaban']
    
    return response  

### Mengecek Akurasi Model

In [12]:
def evaluate_accuracy(test_data):
    test_data['Processed_Pertanyaan'] = test_data['Pertanyaan'].apply(preprocess_text)
    
    correct_predictions = 0  
    
    for i, row in test_data.iterrows():
        processed_query = row['Processed_Pertanyaan']
        expected_answer = row['Jawaban']
        query_tfidf = vectorizer.transform([processed_query])
        cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)
        most_similar_idx = np.argmax(cosine_similarities)
        predicted_answer = faq_data.iloc[most_similar_idx]['Jawaban']
        if predicted_answer == expected_answer:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_data)
    return accuracy

test_data = pd.read_excel('test_data.xlsx')  
accuracy = evaluate_accuracy(test_data)
print(f'Akurasi chatbot: {accuracy * 100:.2f}%')

Akurasi chatbot: 70.37%


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Dataset pertanyaan dan jawaban
data = pd.read_excel("dataset.xlsx")

# Menampilkan nama-nama kolom untuk memastikan format
print(data.columns)

df = pd.DataFrame(data)

# Pertanyaan dari pengguna
user_query = "saya adalah ketua di tim saya, berapa point yang saya dapat jika saya mendapat mendali emas di gemastik?"
# Menghitung TF-IDF untuk dataset dan query
vectorizer = TfidfVectorizer()

# Menggunakan pd.concat() untuk menggabungkan series
tfidf_matrix = vectorizer.fit_transform(pd.concat([df['Pertanyaan'], pd.Series([user_query])]))

# Menghitung Cosine Similarity
cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# Menemukan indeks jawaban yang paling relevan
most_similar_idx = cosine_similarities.argmax()
predicted_answer = df['Jawaban'][most_similar_idx]

# Output jawaban yang diprediksi
print(f'Jawaban yang diprediksi: {predicted_answer}')


Index(['Pertanyaan', 'Jawaban'], dtype='object')
Jawaban yang diprediksi: Beberapa lomba yang dapat Anda ikuti sebagai mahasiswa adalah PIMNAS, GEMASTIK, KRI, LIDM, PILMAPRES, dan KDMI. Lomba-lomba ini mencakup berbagai bidang seperti penelitian, teknologi, inovasi digital, debat, dan lainnya. Pastikan untuk mempersiapkan diri dan mengikuti proses pendaftaran yang telah ditentukan!
